In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
#importing data set
data=pd.read_csv('PCS.csv')

In [6]:
data.dtypes

ID                int64
Date_Time        object
Year              int64
Month            object
Mdate             int64
Day              object
Time              int64
Sensor_ID         int64
Sensor_Name      object
Hourly_Counts    object
dtype: object

In [4]:
data=data[data.Year==2020]
#data.index=range(0,len(data))

In [5]:
data

,ID,Date_Time,Year,Month,Mdate,Day,Time,Sensor_ID,Sensor_Name,Hourly_Counts
2966838,2966839,01/01/2020 12:00:00 AM,2020,January,1,Wednesday,0,4,Town Hall (West),"3,120"
2966839,2966840,01/01/2020 12:00:00 AM,2020,January,1,Wednesday,0,17,Collins Place (South),945
2966840,2966841,01/01/2020 12:00:00 AM,2020,January,1,Wednesday,0,18,Collins Place (North),464
2966841,2966842,01/01/2020 12:00:00 AM,2020,January,1,Wednesday,0,53,Collins St (North),"1,590"
2966842,2966843,01/01/2020 12:00:00 AM,2020,January,1,Wednesday,0,2,Bourke Street Mall (South),725
...,...,...,...,...,...,...,...,...,...,...
3260813,3260814,07/31/2020 11:00:00 PM,2020,July,31,Friday,23,61,Swanston St - RMIT Building 14,21
3260814,3260815,07/31/2020 11:00:00 PM,2020,July,31,Friday,23,63,231 Bourke St,14
3260815,3260816,07/31/2020 11:00:00 PM,2020,July,31,Friday,23,64,Royal Pde-Grattan St,6
3260816,3260817,07/31/2020 11:00:00 PM,2020,July,31,Friday,23,65,Swanston St - City Square,7


In [67]:
new_hour=[]
for i in range(0,len(data)):
    new_hour.append(data.Hourly_Counts[i].replace(',',''))
data.Hourly_Counts=new_hour    

In [71]:
#functions for generating new set of dates without any missing days
def new_dates_set(gap):
    date=[]
    time=min(data_temp['Date_Time'])
    for i in range(0,gap):
        date.append(time+dt.timedelta(hours=i))
    return(date)   

In [72]:
#function to extract months, days, year and time from the new set of generated dates
def extract(df):
    for i in range(0,len(df)):
        df['Year'][i]=date[i].year
        df['Month'][i]=date[i].month
        df['Mdate'][i]=date[i].day
        df['Day'][i]=date[i].weekday()
        df['Time'][i]=date[i].hour
        print(i)
    df['Sensor_ID']=1
    return(df)

In [73]:
#function to match and fill up the hourly pedestrian count in the newly created data frame from the old one
def match(df,data):
    k=0
    for i in range(0,len(data)):
        for j in range(0,100000):
            if(df['Date_Time'][i+k]==data['Date_Time'][i]):
                df['Hourly_Counts'][i+k]=data['Hourly_Counts'][i]
                break
            else:
                k=k+1

In [74]:
# KNN algorithm to fill up missing values
def KNN(data,count=3):
    from sklearn.metrics.pairwise import euclidean_distances
    m=data.isnull().sum()
    mis=[]
    data1=data.copy()
    for i in range(0,data.shape[1]):
        if(m[i]>0):
            mis.append(data.columns[i]) 
        if(len(data.iloc[:,i].value_counts())>50 and data.dtypes[i]=="O"):
            data1.drop(data.columns[i],axis=1,inplace=True)
        if(len(data.iloc[:,i].value_counts())<2):
            data1.drop(data.columns[i],axis=1,inplace=True)
    data1.drop(mis,axis=1,inplace=True)  
    print('Number of columns removed for processing is',len(data.columns)-len(data1.columns))
    for i in range(0,data1.shape[1]):
        if(data1.dtypes[i]=='O'):
            data1.iloc[:,i]=data1.iloc[:,i].astype('category')
    data1=pd.get_dummies(data1,drop_first=True) 
    for i in range(data1.shape[1]):
        data1.iloc[:,i]=(data1.iloc[:,i]-min(data1.iloc[:,i]))/(max(data1.iloc[:,i])-min(data1.iloc[:,i]))  
    for i in mis:
        mi=data[data.loc[:,i].isnull()==True].index
        nu=data[data.loc[:,i].isnull()==False].index
        ary = euclidean_distances(data1.loc[mi,:], data1.loc[nu,:])
        ary=pd.DataFrame(ary)
        ary.index=mi
        ary.columns=nu
        n=0
        if(data.dtypes[data.columns.get_loc(i)]=='O'):
            for j in mi:
                cot=ary.loc[j,:].nsmallest(count)
                cot=cot.index
                val=data.loc[cot,i]
                val=val.mode()
                val=pd.DataFrame(val)
                val=val.iloc[0,0]
                data.loc[j,i]=val
        else:
            for j in mi:
                cot=ary.loc[j,:].nsmallest(count)
                cot=cot.index
                val=data.loc[cot,i]   
                val=val.mean()
                data.loc[j,i]=val
    return(data)  

In [75]:
# algorithm to break down a large dataset into smaller dataframes and pass it into KNN and compile than compile all the smaller data frames
def div(data):
    x=data.shape[0]
    quo=(x//30000)-1
    quo=quo*30000
    data1=[]
    data1=pd.DataFrame(data1)
    for i in range(0,quo,30000):
        print(i/30000)
        data2=KNN(data.iloc[i:i+30000,:],3)
        data1=pd.concat([data1,data2],axis=0)
    data2=KNN(data.iloc[quo:x],3)
    data1=pd.concat([data1,data2],axis=0)
    return(data1)

In [91]:
final_data=pd.DataFrame()

In [92]:
# main algorithm to call function and fill the mising values sensor by sensor
for i in sorted(data.Sensor_ID.unique()):
    print('working on sensor',i)
    data_temp=data[data['Sensor_ID']==i]
    data_temp.index=range(0,len(data_temp))
    data_temp['Date_Time']=pd.to_datetime(data_temp['Date_Time'])
    data_temp=data_temp.sort_values(by=['Date_Time'])
    data_temp.index=range(0,len(data_temp))
    gap=max(data_temp['Date_Time'])-min(data_temp['Date_Time'])
    gap=(gap.days*24)+(gap.seconds/3600)
    print('1')
    date=new_dates_set(int(gap)+1)
    df=pd.DataFrame(index=np.arange(0,gap+1),columns=(['Date_Time','Year','Month','Mdate','Day','Time','Sensor_ID','Hourly_Counts'])) 
    df['Date_Time']=date
    print('2')
    a=[]
    b=[]
    c=[]
    d=[]
    e=[]
    for q in range(0,len(df)):
        a.append(df.loc[q,'Date_Time'].year)
        b.append(df.loc[q,'Date_Time'].month)
        c.append(df.loc[q,'Date_Time'].day)
        d.append(df.loc[q,'Date_Time'].weekday())
        e.append(df.loc[q,'Date_Time'].hour)    
    df['Year']=a
    df['Month']=b
    df['Mdate']=c
    df['Day']=d
    df['Time']=e    
    df['Sensor_ID']=1
    data_temp=data_temp.drop_duplicates(subset=['Date_Time'], keep=False)
    data_temp.index=range(0,len(data_temp))
    print('3')
    k=0
    for w in range(0,len(data_temp)):
        for j in range(0,100000):
            if(df['Date_Time'][w+k]==data_temp['Date_Time'][w]):
                df['Hourly_Counts'][w+k]=data_temp['Hourly_Counts'][w]
                break
            else:
                k=k+1
    df=df.drop(['Date_Time','Sensor_ID'],axis=1)
    df=df.astype('O')
    df['Hourly_Counts']=df['Hourly_Counts'].astype(float)
    print('4')
    semi_final_data=div(df)
    semi_final_data['Date_Time']=date
    semi_final_data['Sensor_ID']=i
#   string=('{n}.csv'.format(n=i))
#   semi_final_data.to_csv(string,index=False)
#   semi_final_data['Hourly_Counts']=semi_final_data['Hourly_Counts'].astype('int64')
#   string=('marked:{n}.csv'.format(n=i))
#   semi_final_data.to_csv(string,index=False)
    final_data=pd.concat([final_data,semi_final_data],axis=0)


working on sensor 1


C:\Users\ausaf\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


1
2
3


C:\Users\ausaf\anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


4
Number of columns removed for processing is 1
working on sensor 2
1
2
3
4
Number of columns removed for processing is 1
working on sensor 3
1
2
3
4
Number of columns removed for processing is 1
working on sensor 4
1
2
3
4
Number of columns removed for processing is 1
working on sensor 5
1
2
3
4
Number of columns removed for processing is 1
working on sensor 6
1
2
3
4
Number of columns removed for processing is 1
working on sensor 7
1
2
3
4
Number of columns removed for processing is 1
working on sensor 8
1
2
3
4
Number of columns removed for processing is 1
working on sensor 9
1
2
3
4
Number of columns removed for processing is 1
working on sensor 10
1
2
3
4
Number of columns removed for processing is 1
working on sensor 11
1
2
3
4
Number of columns removed for processing is 1
working on sensor 12
1
2
3
4
Number of columns removed for processing is 1
working on sensor 14
1
2
3
4
Number of columns removed for processing is 2
working on sensor 15


...

4
Number of columns removed for processing is 2
working on sensor 65


C:\Users\ausaf\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


1
2
3


C:\Users\ausaf\anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


4
Number of columns removed for processing is 1
working on sensor 66
1
2
3
4
Number of columns removed for processing is 1


In [98]:
final_data.index=range(0,len(final_data))

In [100]:
final_data.to_csv('PCS_cleaned.csv',index=False)